# Partition Problem
(see also [Wikipedia](https://en.wikipedia.org/wiki/Partition_problem))

The partition problem for a set of uniformly distributed numbers $\mathcal{S} = \{a_1, ..., a_N\}$ consists of finding two subsets $ \mathcal{S}_{1} \cup \mathcal{S}_2 =  \mathcal{S}$ such that the difference of the sums over the two subsets $\mathcal{S}_{1, 2}$ is as small as possible. The cost function in Ising form can be defined as 
$$
\hat C = -\left(\sum_{i=1}^{N} a_i \hat{Z}_i\right)^2 = \sum_{i<j\leq N} J_{ij} \hat{Z}_i \hat{Z}_j + \mathrm{const.}
$$
with $J_{ij}=-2a_i a_j$. The goal is then to _maximize_ $\hat C$.


In [ ]:
using QAOA, LinearAlgebra
import Random, Distributions

using PyPlot
PyPlot.plt.style.use("paper.mplstyle");

__Defining the problem by hand:__

In [ ]:
N = 4
Random.seed!(1)
a = rand(Distributions.Uniform(0, 1), N)

In [ ]:
J = -2 .* (a * transpose(a))
J[diagind(J)] .= 0.0
J

In [ ]:
p = 4
partition_problem = QAOA.Problem(p, zeros(N), J)

__Using the wrapper function:__

In [ ]:
partition_problem = QAOA.partition_problem(a, num_layers=p)

__Gradient optimization with [Zygote](https://fluxml.ai/Zygote.jl/latest/):__

In [ ]:
learning_rate = 0.05
cost, params, probs = QAOA.optimize_parameters(partition_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]); learning_rate=learning_rate)

__Optimization with [NLopt](https://nlopt.readthedocs.io/en/latest/):__

In [ ]:
cost, params, probs = QAOA.optimize_parameters(partition_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]), :LN_COBYLA)

In [ ]:
xlabels = []
for bstr in digits.(0:2^N-1, base=2, pad=N)
    push!(xlabels, "\$|" * prod([string(b) for b in bstr]) * "\\rangle\$")
end

figure(figsize=(5, 3.2))
ax = subplot(111)
bar(0:2^N-1, probs)
ax.set_xticks(0:2^N-1)
ax.set_xticklabels(xlabels, rotation=90)
minorticks_off()
tight_layout()

In [ ]:
println(a)